# Modeling

In [1]:
# Ignoring warning messages from python
import warnings
warnings.filterwarnings('ignore')

# General use imports
import pandas as pd
import numpy as np

# Specific Modules
import prepare as prep
import json
import os
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


# # Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.graph_objects as go
# import plotly
# import plotly.express as px

In [2]:
df = prep.prep_data()

In [3]:
df.shape

(353, 3)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353 entries, 1 to 475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   repo      353 non-null    object
 1   language  353 non-null    object
 2   content   353 non-null    object
dtypes: object(3)
memory usage: 11.0+ KB


In [5]:
df.head()

,repo,language,content
1,CharlesPikachu/Games,Python,div aligncenter img srcdocslogopng width600 di...
2,channingbreeze/games,JavaScript,### phaser phaserphaserphaserhttpwwwphaserchin...
3,arcxingye/EatKano,JavaScript,p aligncenter hrefhttpsxingyemegameeatkanoimg ...
4,coding-horror/basic-computer-games,C#,### updating first million selling computer bo...
5,rwv/chinese-dos-games,Python,# do do 1898 python 3 python python downloadda...


## Splitting and vectorizing

In [6]:
train, validate, test = prep.split_data(df)

In [7]:
train.shape

(197, 3)

In [8]:
validate.shape

(85, 3)

In [9]:
test.shape

(71, 3)

In [10]:
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(train.content)
y_train = train.language

In [11]:
X_train

<197x13439 sparse matrix of type '<class 'numpy.float64'>'
	with 33640 stored elements in Compressed Sparse Row format>

In [12]:
y_train

115       C++
395      HTML
166        C#
26        C++
162       C++
        ...  
217    Python
156       C++
116    Python
158      Java
153      Java
Name: language, Length: 197, dtype: object

In [13]:
# tfidf = TfidfVectorizer()
X_validate = tfidf.transform(validate.content)
y_validate = validate.language

In [14]:
# tfidf = TfidfVectorizer()
X_test = tfidf.transform(test.content)
y_test = test.language

In [17]:
X_validate.shape, X_train.shape

((85, 13439), (197, 13439))

## Logistic Regression Model

>### Train

In [18]:
model_train = pd.DataFrame(dict(actual=y_train))

In [19]:
model_train.head()

,actual
115,C++
395,HTML
166,C#
26,C++
162,C++


In [20]:
model_train.actual.nunique(), model_train.actual.unique()

(8,
 array(['C++', 'HTML', 'C#', 'C', 'JavaScript', 'Java', 'Python',
        'TypeScript'], dtype=object))

In [21]:
model_train.actual.value_counts()

JavaScript    50
C++           36
Python        33
C#            24
C             18
Java          16
HTML          10
TypeScript    10
Name: actual, dtype: int64

In [22]:
model_train['baseline'] = model_train['actual'] == 'JavaScript'
model_train.head()

,actual,baseline
115,C++,False
395,HTML,False
166,C#,False
26,C++,False
162,C++,False


In [23]:
model_train.baseline.mean()

0.25380710659898476

In [24]:
lm = LogisticRegression().fit(X_train, y_train)

model_train['predicted'] = lm.predict(X_train)

In [25]:
model_train.head()

,actual,baseline,predicted
115,C++,False,C++
395,HTML,False,HTML
166,C#,False,C#
26,C++,False,C++
162,C++,False,C++


In [ ]:
model_train['predicted'] = lm.predict(X_train)

In [26]:
print('Accuracy: {:.2%}'.format(accuracy_score(model_train.actual, model_train.predicted)))
print('------------------')
print('Confusion Matrix')
print(pd.crosstab(model_train.predicted, model_train.actual))
print('------------------')
print('Classification Report')
print('\n')
print(classification_report(model_train.actual, model_train.predicted))

Accuracy: 85.79%
------------------
Confusion Matrix
actual       C  C#  C++  HTML  Java  JavaScript  Python  TypeScript
predicted                                                          
C           11   0    0     0     0           0       0           0
C#           0  24    0     0     0           0       0           0
C++          2   0   35     0     2           0       1           0
HTML         0   0    0     6     0           0       0           0
Java         0   0    0     0     9           0       0           0
JavaScript   5   0    1     4     5          50       0           8
Python       0   0    0     0     0           0      32           0
TypeScript   0   0    0     0     0           0       0           2
------------------
Classification Report


              precision    recall  f1-score   support

           C       1.00      0.61      0.76        18
          C#       1.00      1.00      1.00        24
         C++       0.88      0.97      0.92        36
       

>### Validate

In [31]:
model_train.head()

,actual,baseline,predicted
115,C++,False,C++
395,HTML,False,HTML
166,C#,False,C#
26,C++,False,C++
162,C++,False,C++


In [26]:
print('Accuracy: {:.2%}'.format(accuracy_score(model_train.actual, model_train.predicted)))
print('------------------')
print('Confusion Matrix')
print(pd.crosstab(model_train.predicted, model_train.actual))
print('------------------')
print('Classification Report')
print('\n')
print(classification_report(model_train.actual, model_train.predicted))

Accuracy: 85.79%
------------------
Confusion Matrix
actual       C  C#  C++  HTML  Java  JavaScript  Python  TypeScript
predicted                                                          
C           11   0    0     0     0           0       0           0
C#           0  24    0     0     0           0       0           0
C++          2   0   35     0     2           0       1           0
HTML         0   0    0     6     0           0       0           0
Java         0   0    0     0     9           0       0           0
JavaScript   5   0    1     4     5          50       0           8
Python       0   0    0     0     0           0      32           0
TypeScript   0   0    0     0     0           0       0           2
------------------
Classification Report


              precision    recall  f1-score   support

           C       1.00      0.61      0.76        18
          C#       1.00      1.00      1.00        24
         C++       0.88      0.97      0.92        36
       